# Import packages

In [8]:
import pandas as pd
from teradataml import *

from moduels.db import TeradataDatabase
from constants import TEST_DATA_PATH

## Load database

In [9]:
td_db = TeradataDatabase()
td_db.connect()

## Load data to database

In [12]:
df = pd.read_csv(TEST_DATA_PATH)
df.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,Americana Okra zero 400 gm,Vegetables & Fruits,Americana,400gm,1,NaN,NaN,NaN,كيس,gm
1,ليمون اداليا 500 جم,"Tins, Jars & Packets",NaN,500جم,1,NaN,NaN,NaN,علبة,جم
2,صلصه هاينز برطمان خصم عرض,"Tins, Jars & Packets",هاينز,NaN,1,NaN,NaN,NaN,علبة,NaN
3,Dasani water 330ML,Water,Dasani,330مل,1,NaN,NaN,NaN,زجاجة,مل
4,بودرة عصير أناناس من سورس، 900 جم,Soft Drinks & Juices,سورس,900جم,1,NaN,NaN,NaN,عبوة,جم


In [ ]:
df = df[["Item_Name", "Brand"]]
df.rename(columns={"Item_Name": "product_name", "Brand": "brand_name"}, inplace=True)

,product_name,brand_name
0,Americana Okra zero 400 gm,Americana
1,ليمون اداليا 500 جم,NaN
2,صلصه هاينز برطمان خصم عرض,هاينز
3,Dasani water 330ML,Dasani
4,بودرة عصير أناناس من سورس، 900 جم,سورس
...,...,...
4768,جهينه كوكتيل 1ل,جهينه
4769,دبل ديرى مشروب بنكهه ليمون 275مل,دبل ديرى
4770,Galaxy Smooth Milk Chocolate 24 x 36g,Galaxy
4771,Hot Ketchup Squeeze – 320g,NaN


In [17]:
from teradataml import DataFrame
tdf = DataFrame(df)
tdf

TeradataMlException: [Teradata][teradataml](TDML_2020) Failed to copy dataframe to Teradata Vantage.[Version 20.0.0.34] [Session 1105] [Teradata SQL Driver] [Error 502] Batch row 2 bound parameter 2 type FLOAT (480) differs from batch row 1 type VARCHAR (448)
 at gosqldriver/teradatasql.formatError ErrorUtil.go:83
 at gosqldriver/teradatasql.(*teradataConnection).makeDriverError ErrorUtil.go:150
 at gosqldriver/teradatasql.(*TeradataRows).validateBatchValues TeradataRows.go:1697
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:667
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:823
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1786
 at database/sql.withLock sql.go:3574
 at database/sql.(*DB).queryDC sql.go:1781
 at database/sql.(*Conn).QueryContext sql.go:2037
 at main.createRows goside.go:1082
 at main.goCrea

In [6]:
tdf = DataFrame(in_schema("amurd", "products"))
tdf.head()

product_id,product_name,brand_name


In [7]:
td_db.disconnect()